In [1]:
from Py6S import *
import numpy as np
%matplotlib inline
import ipywidgets as widgets
from matplotlib import pyplot as plt

In [2]:
s = SixS()
#s.atmos_profile = AtmosProfile.PredefinedType(AtmosProfile.Tropical)

s.altitudes.set_sensor_satellite_level()
s.atmos_profile = AtmosProfile.PredefinedType(AtmosProfile.Tropical)
#s.ground_reflectance = GroundReflectance.HomogeneousLambertian(GroundReflectance.GreenVegetation)
s.geometry = Geometry.User()
s.geometry.from_time_and_location(-35.0481, 150.7447, "06:00:00", 0, 0) #% 11am AEDT

wv, res = SixSHelpers.Wavelengths.run_wavelengths(s,np.arange(0.400, 1, 0.01), output_name='pixel_radiance')
phots=res/(1.98644582e-25/(wv*1e-6))
SixSHelpers.Wavelengths.plot_wavelengths(wv, phots, "rad at satel. level (photons/s/m^2/sr/mic)")

Running for many wavelengths - this may take a long time


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
loopfnum=[]
loopgsd=[]
loopsn=[]
for loopefl in range(50, 800, 10):
    apeture=40e-3
    efl=loopefl*1e-3

    fnum=efl/apeture

    pixelsize=5e-6 #m

    pixelFOV=pixelsize/efl

    int_time=2e-3 # integrations time, s
    dlam=0.015 #um

    QE=0.7 # detector quantum efficeny
    trans=0.8 # trans. of optics

    SLDangle=apeture**2 * np.pi / (4*efl**2)

    A=pixelsize**2

    #photons/s/m^2/sr/mic * losses
    captured_phots=phots * int_time * A * SLDangle * dlam * trans * QE

    #SixSHelpers.Wavelengths.plot_wavelengths(wv, captured_phots, "rad at satel. level collected (photons per pixel)")
    #SixSHelpers.Wavelengths.plot_wavelengths(wv, captured_phots/np.sqrt(captured_phots), "S/N")
    loopfnum.append(fnum)
    loopgsd.append(550e3/efl *2 *pixelsize)
    loopsn.append(np.nanmean(np.sqrt(captured_phots)))
    
    #     print("F/#: {}".format(fnum))
    #     print("Resolution Element Size: {} m, for EFL: {} mm".format(550e3/efl *2 *pixelsize,efl*1e3))
    #     print("Mean S/N: {}".format(np.nanmean(np.sqrt(captured_phots))))

In [19]:
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel('F/#')
ax1.set_ylabel('S/N', color=color)
ax1.plot(loopfnum, loopsn, color=color)
ax1.tick_params(axis='y', labelcolor=color)
plt.grid(True)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('GSD (m)', color=color)  # we already handled the x-label with ax1
ax2.plot(loopfnum, loopgsd, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …